In [1]:
import nltk
from nltk import FreqDist
import string
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
%matplotlib inline

In [2]:
def get_stream_details():
    print("Reading the stream details...")
    complete_stream_details_df = pd.read_csv("H:\\TeamStreamz_IW\\code\\data\\card_module_details_content_extracted.csv", encoding="ISO-8859-1")
    if complete_stream_details_df is not None:
        complete_stream_details_dict = {}
        _stream_id_stream_title_dict = {}
        for _, row in complete_stream_details_df.iterrows():
            
            stream_id = str(row["DECKID"])
            stream_title = str(row["DECKNAME"])
            row_content = str(row["HTML_CONTENT"])

            # TODO: add the card title and the module name to the content on which the tags can be generated
            card_title =str(row["CARDTITLE"])
            module_name = str(row["MODULENAME"])
            
            if row_content and "nan" not in row_content:
                # if the stream ID already exists in the dictionary
                if complete_stream_details_dict.get(stream_id):
                    existing_content = complete_stream_details_dict[stream_id]
                    new_content = existing_content + "\n" + row_content.strip()
                    complete_stream_details_dict[stream_id] = new_content
                else:
                    complete_stream_details_dict[stream_id] = row_content.strip()
                    _stream_id_stream_title_dict[stream_id] = stream_title
        
        return complete_stream_details_dict

In [4]:
stream_details_dict = get_stream_details()
#stream_details_dict

Reading the stream details...


In [5]:
# what is the content length
stream_details_df = pd.DataFrame(list(stream_details_dict.items()), columns = ["StreamID", "Content"])
stream_details_df["ContentLength"] = stream_details_df["Content"].str.len()
stream_details_df.head()

,StreamID,Content,ContentLength
0,163,TXmAk2KZAy4\nNMeUjebo1Ac\nEEuTxFhp3go\nCastrol...,801
1,419,TXmAk2KZAy4\nNMeUjebo1Ac\nEEuTxFhp3go\nCastrol...,813
2,507,"wBYKUgUyGWc\nA team of world-class drivers, po...",181
3,199,Castrol EDGE is Castrols flagship power bran...,2550
4,201,"Charles Cheers Wakefield, Castrols founder,...",1662


In [15]:
from nltk import ngrams
# convert case, remove punctuations
# stem the words
stop = stopwords.words('english')
snowball = nltk.SnowballStemmer('english')

def preprocess(toks):
    toks = [ t.lower() for t in toks if t not in string.punctuation ]
    toks = [t for t in toks if t not in stop ]
    toks = [ snowball.stem(t) for t in toks ]
#   toks = [ wnl.lemmatize(t) for t in toks ]
    toks_clean = [ t for t in toks if len(t) >= 3 ]
    return toks_clean

all_streams_cleaned_tokens = []
stream_ids = list(stream_details_df["StreamID"])
i=0
combined = []
for _, row in stream_details_df.iterrows():
    stream_content = row["Content"]
    stream_content_tokens = nltk.word_tokenize(stream_content)
    stream_cleaned_tokens = preprocess(stream_content_tokens)
    all_streams_cleaned_tokens.append(stream_cleaned_tokens)
#     print(stream_cleaned_tokens)
    n_grams = ngrams(stream_cleaned_tokens, 3)
    #num_ngrams = len(list(n_grams))
    #print(num_ngrams)
    #combined = "_".join(list(n_grams))
    #print(list(n_grams))
    
    for n_gram in n_grams:
        combined_val = "_".join(n_gram)
        combined.append(combined_val)
    #print(combined)
    #i+=1
#     for grams in n_grams:
#         print(list(grams))

print("Original number of 3 tuples: " + str(len(combined)))
# to remove duplicates
combined = set(combined)
#print(combined)
print("After duplicate removal: " + str(len(combined)))

G:\anaconda\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: generator 'ngrams' raised StopIteration


Original number of 3 tuples: 12149
After duplicate removal: 4831
